In [6]:
!pip install tensorflow paddlepaddle paddleocr
!apt-get install -y libglib2.0-0 libsm6 libxrender1 libxext6


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1build2).
libxext6 is already the newest version (2:1.3.4-1build1).
libxrender1 is already the newest version (1:0.9.10-1build4).
libglib2.0-0 is already the newest version (2.72.4-0ubuntu2.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from paddleocr import PaddleOCR
import re
from PIL import Image

class Solution:
    def __init__(self):
        """
        Initialize your data structures here.
        """
        # Load VGG16 model pre-trained on ImageNet
        base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
        self.model = Model(inputs=base_model.input, outputs=base_model.output)
        self.ocr = PaddleOCR(use_angle_cls=True, lang='en')

    def preprocess_image(self, img_path):
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array

    def extract_features(self, img_path):
        img_array = self.preprocess_image(img_path)
        features = self.model.predict(img_array)
        return features

    def calculate_similarity(self, main_image_path, test_image_path1, test_image_path2):
        main_features = self.extract_features(main_image_path)
        test1_features = self.extract_features(test_image_path1)
        test2_features = self.extract_features(test_image_path2)

        # Compute cosine similarity
        similarity1 = cosine_similarity(main_features, test1_features)[0][0]
        similarity2 = cosine_similarity(main_features, test2_features)[0][0]

        percentage1 = similarity1 * 100
        percentage2 = similarity2 * 100

        return [percentage1, percentage2]

    def process_image_sets(self, base_path, set_number):
        main_image_path = f'{base_path}/Set{set_number}/Image.png'
        test_image_path1 = f'{base_path}/Set{set_number}/Test1.png'
        test_image_path2 = f'{base_path}/Set{set_number}/Test2.png'

        similarity_percentages = self.calculate_similarity(main_image_path, test_image_path1, test_image_path2)
        return similarity_percentages

    def extract_numerical_value(self, text):
        pattern = r'[-+]?\d{1,3}(?:,\d{3})*(?:\.\d+)?'
        match = re.search(pattern, text)
        if match:
            return float(match.group().replace(',', ''))
        else:
            return None

    def find_nearest_numerical_value(self, target_word, ocr_result, img_width, img_height, max_distance):
        target_box = None
        for (box, text) in zip(ocr_result['boxes'], ocr_result['texts']):
            if target_word in text:
                target_box = box
                break

        if target_box is None:
            return ""

        target_x, target_y = target_box[0][0], target_box[0][1]
        target_center = np.array([target_x + (target_box[1][0] - target_x) / 2,
                                  target_y + (target_box[2][1] - target_y) / 2])

        closest_value = None
        closest_distance = float('inf')

        for box, text in zip(ocr_result['boxes'], ocr_result['texts']):
            numerical_value = self.extract_numerical_value(text)
            if numerical_value is not None:
                box_x, box_y = box[0][0], box[0][1]
                box_center = np.array([box_x + (box[1][0] - box_x) / 2,
                                       box_y + (box[2][1] - box_y) / 2])

                distance = np.linalg.norm(target_center - box_center)

                if distance < closest_distance and distance <= max_distance:
                    closest_distance = distance
                    closest_value = numerical_value

        return closest_value if closest_value is not None else ""

    def get_answer(self, problem):
        base_path = '/content/drive/MyDrive/Problems'
        if 'Set 8' in problem:
            test_image_path1 = f'{base_path}/Set8/Test1.png'
            test_image_path2 = f'{base_path}/Set8/Test2.png'
            first_nearest = self.process_for_last(test_image_path1, 'TOTAL WIN')
            second_nearest = self.process_for_last(test_image_path2, 'TOTAL WIN')
            return [first_nearest, second_nearest]

        elif 'Set 9' in problem:
            test_image_path1 = f'{base_path}/Set9/Test1.png'
            test_image_path2 = f'{base_path}/Set9/Test2.png'
            first_nearest = self.process_for_last(test_image_path1, 'BET')
            second_nearest = self.process_for_last(test_image_path2, 'BET')
            return [first_nearest, second_nearest]

        else:
            set_number = int(problem.split('Set')[1])
            return self.process_image_sets(base_path, set_number)

    def process_for_last(self, img_path, target_word):
        img = Image.open(img_path)
        ocr_result = self.ocr.ocr(img_path, cls=True)

        parsed_result = {
            'boxes': [],
            'texts': [],
            'scores': []
        }

        for line in ocr_result:
            for word_info in line:
                parsed_result['boxes'].append(word_info[0])
                parsed_result['texts'].append(word_info[1][0])
                parsed_result['scores'].append(word_info[1][1])

        max_distance = 150  # Increased range
        nearest_value = self.find_nearest_numerical_value(target_word, parsed_result, img.width, img.height, max_distance)
        return nearest_value


In [4]:
from google.colab import drive
drive.mount('/content/drive')

# Define base path
base_path = '/content/drive/MyDrive/Problems'


Mounted at /content/drive


In [5]:
solution = Solution()

# Process Sets 1 to 7 for image similarity
for i in range(1, 8):
    similarity_percentages = solution.process_image_sets(base_path, i)
    print(f"Set {i}: {similarity_percentages}")

# Process Sets 8 and 9 for OCR
total_win_amounts = solution.get_answer('Set 8')
bet_amounts = solution.get_answer('Set 9')

print(f"Set 8: TOTAL WIN amounts are {total_win_amounts}")
print(f"Set 9: BET amounts are {bet_amounts}")


58889256/58889256 [==============================] - 0s 0us/step
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:15<00:00, 258kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:17<00:00, 591kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:13<00:00, 161kiB/s]

[2024/06/22 08:58:02] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

1/1 [==============================] - 1s 556ms/step
Set 1: [98.78835082054138, 88.98774981498718]
1/1 [==============================] - 1s 547ms/step
Set 2: [60.9222948551178, 70.34170031547546]
1/1 [==============================] - 1s 1s/step
Set 3: [40.74313044548035, 37.67572045326233]
1/1 [==============================] - 1s 534ms/step
Set 4: [20.9756076335907, 20.54901123046875]
1/1 [==============================] - 1s 533ms/step
Set 5: [37.567877769470215, 38.096582889556885]
1/1 [==============================] - 1s 533ms/step
Set 6: [30.858635902404785, 31.758710741996765]
1/1 [==============================] - 1s 1s/step
Set 7: [62.63188123703003, 59.06163454055786]
[2024/06/22 08:58:32] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.35036134719848633
[2024/06/22 08:58:32] ppocr DEBUG: cls num  : 20, elapsed : 0.09523558616638184
[2024/06/22 08:58:33] ppocr DEBUG: rec_res num  : 20, elapsed : 1.1977431774139404
[2024/06/22 08:58:34] ppocr DEBUG: dt_boxes num : 17, elapsed : 